---

## EVCS POI Candidate Selection

This section outlines the process of selecting POI (Point of Interest) candidates categorized by region.

- The above approach represents the process of selecting POI candidates categorized by region. POI filtering was applied when selecting POI candidates for all regions.
  
  - Before filtering, OSM_POI data was imported, and POIs were extracted for each categorized region (Atlanta, Suburban, Rural) using GIS tools.
  
  - For each extracted POI candidate, filtering was performed using the 'fclass' column, which contains information about building usage, utilizing GIS tools.

<br>

---

### Summary

- **Level 2 POI Candidate Selection**: Georgia, using GIS tools & filtering.  
- **DCFC POI Candidate Selection**: Georgia, using GIS tools & filtering.  
<br>
---


In [ ]:
import yaml
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
import random
from shapely.geometry import box
from tqdm import tqdm

In [ ]:
# Load the YAML file with fclass categories
yaml_file_path = 'poi_filtering.yaml'

with open(yaml_file_path, 'r') as file:
    fclass_data = yaml.safe_load(file)

# Define the region and file paths
region = 'rural'
gpkg_file_path = "Path To your Rural Region POI"
output_path = "rural_poi_candidate.gpkg"

# Load the fclass categories for the region from the YAML data
if region in fclass_data['candidate']:
    selected_fclasses = fclass_data['candidate'][region]
else:
    raise ValueError(f"Region '{region}' not found in the YAML file.")

# Read the GPKG file
poi_gdf = gpd.read_file(gpkg_file_path)

# Filter the POI data based on the selected fclass categories
selected_poi_gdf = poi_gdf[poi_gdf['fclass'].isin(selected_fclasses)]

# Get the count of filtered features
selected_feature_count = selected_poi_gdf.shape[0]
print(f"Selected feature count for {region}: {selected_feature_count}")

# Save the filtered data to a new GPKG file
selected_poi_gdf.to_file(output_path, driver='GPKG')
print(f"Filtered POI data saved for {region} at {output_path}")


## Case 5: Rural Level 2 EVCS Charger Location Selection  
## Case 6: Rural DCFC EVCS Charger Location Selection

For **Case 5** and **Case 6**, which pertain to **Rural areas**, the demand for fast charging (DCFC) is typically low. Most charging needs in rural areas are met with slower chargers (Level 2), and thus, the allocated **capacity** for EVCS in these regions is not high.

When installing **EVCS** and **DCFC** chargers in rural areas, a thorough **POI filtering process** was conducted, followed by the use of the **Demand Map** to determine the optimal locations for charger deployment. This ensured that chargers were placed where they would meet the actual demand of the rural population while making efficient use of available capacity.

### Algorithm Summary:

1. **Load Data**: Load rural polygons, POI candidate data, and demand map (GeoTIFF).

2. **Convert Demand Pixels to Polygons**: Extract demand values from the GeoTIFF file and convert them into polygons.

3. **Spatial Join**: Perform a spatial join between the POI data and demand polygons to assign demand values to POIs.

4. **POI Selection**:
   - Filter POIs within each rural polygon.
   - Select the POI with the highest demand, prioritizing those in **priority categories**.
   - If no priority category POI exists, randomly select the highest demand POI.

5. **DCFC & Level2 Type Are Same Algorithm in Here** :

In [ ]:
# File paths
poi_file_path = 'Path to your rural_poi_candidate.gpkg'  # POI candidate data
polygon_file_path = "Path to your Rural Polygon.gpkg"  # Rural polygons
demand_tif_file_path = 'Path to your Demand map'  # GeoTIFF demand map
output_file_path = "Path to your Output Path"  # Output file

# Parameters
priority_categories = ['department_store', 'town_hall', 'hospital', 'market_place', 
                       'mall', 'public_building']  # Priority POI categories

# 1. Load the data
polygon_gdf = gpd.read_file(polygon_file_path)  # Rural polygons
poi_gdf = gpd.read_file(poi_file_path)  # POI candidate data

# 2. Read the demand data from the GeoTIFF and convert the demand pixels to polygons
with rasterio.open(demand_tif_file_path) as src:
    affine = src.transform
    demand_data = src.read(1)  # Read the first band (demand values)
    
    # Extract rows and columns with demand values greater than 0
    rows, cols = np.where(demand_data > 0)
    
    # Generate demand polygons and corresponding demand values
    demand_polygons = [box(*src.xy(row, col), *src.xy(row + 1, col + 1)) for row, col in zip(rows, cols)]
    demand_values = [demand_data[row, col] for row, col in zip(rows, cols)]

# 3. Convert demand polygons into a GeoDataFrame
demand_gdf = gpd.GeoDataFrame({'demand': demand_values, 'geometry': demand_polygons}, crs="EPSG:3857")

# 4. Perform a spatial join to associate demand values with POIs
poi_with_demand_gdf = gpd.sjoin(poi_gdf, demand_gdf, how='left', predicate='within')

# 5. Select the POI with the highest demand in each rural polygon
selected_pois_list = []

for _, rural_polygon in tqdm(polygon_gdf.iterrows(), total=len(polygon_gdf), desc="Processing rural polygons"):
    # Filter POIs within the current polygon and those with demand values
    pois_in_polygon = poi_with_demand_gdf[poi_with_demand_gdf.within(rural_polygon.geometry)]
    
    if pois_in_polygon.empty:
        continue  # Skip to the next polygon if no POIs are found

    # Select POIs with the highest demand value
    max_demand_value = pois_in_polygon['demand'].max()
    top_demand_pois = pois_in_polygon[pois_in_polygon['demand'] == max_demand_value]
    
    # Select POIs from priority categories if available
    prioritized_pois = top_demand_pois[top_demand_pois['fclass'].isin(priority_categories)]
    
    if not prioritized_pois.empty:
        selected_poi = prioritized_pois.sample(n=1)  # Randomly select one POI from the priority categories
    else:
        selected_poi = top_demand_pois.sample(n=1)  # Select one POI from the top demand POIs

    selected_pois_list.append(selected_poi)

# 6. Save the selected POIs to a new GeoDataFrame
if selected_pois_list:
    selected_pois_gdf = gpd.GeoDataFrame(pd.concat(selected_pois_list, ignore_index=True), crs='EPSG:3857')
    
    # Ensure column names are within 10 characters (ESRI Shapefile limit)
    selected_pois_gdf.columns = [str(col)[:10] for col in selected_pois_gdf.columns]
    
    # Save the selected POIs to a GeoPackage file
    selected_pois_gdf.to_file(f"{output_file_path}rural_dcfc_evcs.gpkg", driver="GPKG")
    print(f"Selected POIs saved to: {output_file_path}rural_dcfc_evcs.gpkg")
else:
    print("No POIs were selected.")